In [1]:
from torchvision import transforms
from PIL import Image
import os
import torch
import glob
import os
from torchvision.datasets import MNIST, CIFAR10, FashionMNIST, ImageFolder
import numpy as np

import torch
#from dataset import get_data_transforms, load_data
from torchvision.datasets import ImageFolder
import numpy as np
from torch.utils.data import DataLoader
#from resnet import resnet18, resnet34, resnet50, wide_resnet50_2
#from de_resnet import de_resnet18, de_resnet50, de_wide_resnet50_2

from torch.nn import functional as F
from sklearn.metrics import roc_auc_score
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from skimage import measure
import pandas as pd
from numpy import ndarray
from statistics import mean
from scipy.ndimage import gaussian_filter
from sklearn import manifold
from matplotlib.ticker import NullFormatter
from scipy.spatial.distance import pdist
import matplotlib
import pickle
import random

import torch
from torch import Tensor
import torch.nn as nn
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional

# helpers

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Data

### get data transform

In [3]:
def get_data_transforms(size, isize):
    
    mean_train = [0.485, 0.456, 0.406]
    std_train = [0.229, 0.224, 0.225]
    
    data_transforms = transforms.Compose([
                      transforms.Resize((size, size)),
                      transforms.ToTensor(),
                      transforms.CenterCrop(isize),
                      transforms.Normalize(mean=mean_train, std=std_train)])
    
    gt_transforms = transforms.Compose([
                    transforms.Resize((size, size)),
                    transforms.CenterCrop(isize),
                    transforms.ToTensor()])
    
    data_transforms_train = transforms.Compose([
                      transforms.Resize((size, size)),
                      transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0),
                      #transforms.RandomAffine(degrees = 30, translate=(0.,0.), scale=None, shear=None),
                      #transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomVerticalFlip(p=0.5),
                      transforms.ToTensor(),
                      transforms.CenterCrop(isize),
                      transforms.Normalize(mean=mean_train, std=std_train)])
    
    return data_transforms, gt_transforms, data_transforms_train

def get_data_transforms_clean(size, isize):

    
    data_transforms = transforms.Compose([
                      transforms.Resize((size, size)),
                      transforms.ToTensor(),
                      transforms.CenterCrop(isize)])
    
    gt_transforms = transforms.Compose([
                    transforms.Resize((size, size)),
                    transforms.CenterCrop(isize),
                    transforms.ToTensor()])
    
    data_transforms_train = transforms.Compose([
                      transforms.Resize((size, size)),
                      #transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0),
                      transforms.RandomAffine(degrees = 30, translate=(0.,0.), scale=None, shear=None),
                      #transforms.RandomHorizontalFlip(p=0.5),
                      #transforms.RandomVerticalFlip(p=0.5),
                      transforms.ToTensor(),
                      transforms.CenterCrop(isize)
    ])
    
    return data_transforms, gt_transforms, data_transforms_train

### MTVecDataset

In [4]:
class MVTecDataset(torch.utils.data.Dataset):
    
    
    def __init__(self, root, transform, gt_transform, phase):
        
        if phase == 'train':
            self.img_path = os.path.join(root, 'train')
        else:
            self.img_path = os.path.join(root, 'test')
            self.gt_path = os.path.join(root, 'ground_truth')
            
        self.transform = transform
        self.gt_transform = gt_transform
        
        # load dataset
        self.img_paths, self.gt_paths, self.labels, self.types = self.load_dataset()  # self.labels => good : 0, anomaly : 1

        
        
    def load_dataset(self):

        img_tot_paths = []
        gt_tot_paths = []
        tot_labels = []
        tot_types = []

        defect_types = os.listdir(self.img_path)

        for defect_type in defect_types:
            if defect_type == 'good':
                img_paths = glob.glob(os.path.join(self.img_path, defect_type) + "/*.png")
                img_tot_paths.extend(img_paths)
                gt_tot_paths.extend([0] * len(img_paths))
                tot_labels.extend([0] * len(img_paths))
                tot_types.extend(['good'] * len(img_paths))
            else:
                img_paths = glob.glob(os.path.join(self.img_path, defect_type) + "/*.png")
                gt_paths = glob.glob(os.path.join(self.gt_path, defect_type) + "/*.png")
                img_paths.sort()
                gt_paths.sort()
                img_tot_paths.extend(img_paths)
                gt_tot_paths.extend(gt_paths)
                tot_labels.extend([1] * len(img_paths))
                tot_types.extend([defect_type] * len(img_paths))

        assert len(img_tot_paths) == len(gt_tot_paths), "Something wrong with test and ground truth pair!"

        return img_tot_paths, gt_tot_paths, tot_labels, tot_types

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path, gt, label, img_type = self.img_paths[idx], self.gt_paths[idx], self.labels[idx], self.types[idx]
        #print(img_path)
        img = Image.open(img_path).convert('RGB')
        
        #print("data")
        #plt.imshow(img)
        #plt.show()
        
        img = self.transform(img)
        if gt == 0:
            gt = torch.zeros([1, img.size()[-2], img.size()[-2]])
        else:
            gt = Image.open(gt)
            gt = self.gt_transform(gt)

        assert img.size()[1:] == gt.size()[1:], "image.size != gt.size !!!"

        return img, gt, label, img_type


# check augmentation

In [5]:
_classes_ = ['carpet', 'bottle', 'hazelnut', 'leather', 'cable', 'capsule', 'grid', 'pill','transistor', 'metal_nut', 'screw','toothbrush', 'zipper', 'tile', 'wood']
_class_ = _classes_[np.random.randint(0,len(_classes_))]
image_size = 256

_class_ = 'carpet'

print(_class_)

data_transforms, gt_transform, data_transforms_train = get_data_transforms_clean(image_size, image_size)
train_path = '/kaggle/input/mvtec_anomaly_detection/' + _class_ 

data = MVTecDataset(root=train_path, transform=data_transforms_train, gt_transform=gt_transform, phase="train")
data2 = MVTecDataset(root=train_path, transform=data_transforms, gt_transform=gt_transform, phase="train")

ii = np.random.randint(0,10)

img, gt, label,_ = data.__getitem__(ii)
img2, gt, label,_ = data2.__getitem__(ii)

f, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].imshow(img.permute(1, 2, 0))
ax[1].imshow(img2.permute(1, 2, 0))
plt.show() 

carpet


### load data

In [6]:
def load_data(dataset_name='mnist',normal_class=0,batch_size='16'):

    
    if dataset_name == 'cifar10':
        
        img_transform = transforms.Compose([
                transforms.Resize((32, 32)),
                #transforms.CenterCrop(28),
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
                ])

        os.makedirs("./Dataset/CIFAR10/train", exist_ok=True)
        dataset = CIFAR10('./Dataset/CIFAR10/train', train=True, download=True, transform=img_transform)
        print("Cifar10 DataLoader Called...")
        print("All Train Data: ", dataset.data.shape)
        dataset.data = dataset.data[np.array(dataset.targets) == normal_class]
        dataset.targets = [normal_class] * dataset.data.shape[0]
        print("Normal Train Data: ", dataset.data.shape)

        os.makedirs("./Dataset/CIFAR10/test", exist_ok=True)
        test_set = CIFAR10("./Dataset/CIFAR10/test", train=False, download=True, transform=img_transform)
        print("Test Train Data:", test_set.data.shape)

        
        
    elif dataset_name == 'mnist':
        
        img_transform = transforms.Compose([
                        transforms.Resize((32, 32)),
                        transforms.ToTensor()
                         ])

        os.makedirs("./Dataset/MNIST/train", exist_ok=True)
        dataset = MNIST('./Dataset/MNIST/train', train=True, download=True, transform=img_transform)
        print("MNIST DataLoader Called...")
        print("All Train Data: ", dataset.data.shape)
        dataset.data = dataset.data[np.array(dataset.targets) == normal_class]
        dataset.targets = [normal_class] * dataset.data.shape[0]
        print("Normal Train Data: ", dataset.data.shape)

        os.makedirs("./Dataset/MNIST/test", exist_ok=True)
        test_set = MNIST("./Dataset/MNIST/test", train=False, download=True, transform=img_transform)
        print("Test Train Data:", test_set.data.shape)

        
        
    elif dataset_name == 'fashionmnist':
        
        img_transform = transforms.Compose([
                        transforms.Resize((32, 32)),
                        transforms.ToTensor()
                        ])

        os.makedirs("./Dataset/FashionMNIST/train", exist_ok=True)
        dataset = FashionMNIST('./Dataset/FashionMNIST/train', train=True, download=True, transform=img_transform)
        print("FashionMNIST DataLoader Called...")
        print("All Train Data: ", dataset.data.shape)
        dataset.data = dataset.data[np.array(dataset.targets) == normal_class]
        dataset.targets = [normal_class] * dataset.data.shape[0]
        print("Normal Train Data: ", dataset.data.shape)

        os.makedirs("./Dataset/FashionMNIST/test", exist_ok=True)
        test_set = FashionMNIST("./Dataset/FashionMNIST/test", train=False, download=True, transform=img_transform)
        print("Test Train Data:", test_set.data.shape)


    elif dataset_name == 'retina':
        
        data_path = 'Dataset/OCT2017/train'

        orig_transform = transforms.Compose([
                        transforms.Resize([128, 128]),
                        transforms.ToTensor()
                    ])

        dataset = ImageFolder(root=data_path, transform=orig_transform)

        test_data_path = 'Dataset/OCT2017/test'
        test_set = ImageFolder(root=test_data_path, transform=orig_transform)

    else:
        raise Exception(
            "You enter {} as dataset, which is not a valid dataset for this repository!".format(dataset_name))

    train_dataloader = torch.utils.data.DataLoader(
                                                    dataset,
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                )
    
    test_dataloader = torch.utils.data.DataLoader(
                                                    test_set,
                                                    batch_size=1,
                                                    shuffle=False,
                                                )

    return train_dataloader, test_dataloader

# Resnet

In [7]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'wide_resnet50_2', 'wide_resnet101_2']


model_urls = { 'resnet18': 'https://download.pytorch.org/models/resnet18-f37072fd.pth', 'resnet34': 'https://download.pytorch.org/models/resnet34-b627a593.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-0676ba61.pth', 'resnet101': 'https://download.pytorch.org/models/resnet101-63fe2227.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-394f9c45.pth',  'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth', 'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


In [8]:
bias = False
bias_de = True

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=bias, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=bias)

def deconv2x2(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.ConvTranspose2d(in_planes, out_planes, kernel_size=2, stride=stride, groups=groups, bias=bias_de, dilation=dilation)

### Encoder Resnet

In [9]:
### not used nur bei Tyoe Unio:
class Bottleneck(nn.Module):
    
    expansion: int = 4

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        
        #raise Exception("FUCK BN used")
        
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [10]:
# encoder
class BasicBlock(nn.Module):
    
    expansion: int = 1

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, 
                 groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block: Type[Union[BasicBlock, Bottleneck]], layers: List[int], num_classes: int = 1000, zero_init_residual: bool = False, groups: int = 1, 
                 width_per_group: int = 64, replace_stride_with_dilation: Optional[List[bool]] = None,norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None or a 3-element tuple, got {}".format(replace_stride_with_dilation))
            
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        #self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) #. oder
        
        self.layer1 = self._make_layer(block,  64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

                    
    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int, stride: int = 1, dilate: bool = False) -> nn.Sequential:
        
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(conv1x1(self.inplanes, planes * block.expansion, stride), norm_layer(planes * block.expansion),)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups = self.groups, base_width = self.base_width, dilation = self.dilation, norm_layer = norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
    
        #print("input: ", x.shape)
        x = self.conv1(x)
        #print("input: ", x.shape)
        x = self.bn1(x)
        #print("input: ", x.shape)
        x = self.relu(x)
        #print("input: ", x.shape)
        x = self.maxpool(x)
        #print("input: ", x.shape)

        feature_a = self.layer1(x)
        #print("a: ", feature_a.shape)
        feature_b = self.layer2(feature_a)
        #print("b: ", feature_b.shape)
        feature_c = self.layer3(feature_b)
        #print("c: ", feature_c.shape)
        feature_d = self.layer4(feature_c)
        #print("d: ", feature_d.shape)

        return [feature_a, feature_b, feature_c]

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(arch: str, block: Type[Union[BasicBlock, Bottleneck]], layers: List[int], pretrained: bool, progress: bool, **kwargs: Any) -> ResNet:
    
    model = ResNet(block, layers, **kwargs)
    
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress = progress)
        model.load_state_dict(state_dict)
        
    return model

In [11]:
 class AttnBottleneck(nn.Module):
    
    expansion: int = 4

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None, attention: bool = True,) -> None:
        
        super(AttnBottleneck, self).__init__()
        self.attention = attention
        #print("Attention:",self.attention)
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        #self.cbam = GLEAM([int(planes * self.expansion/4),
        #                   int(planes * self.expansion//2),
        #                   planes * self.expansion], 16)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:

        identity = x


        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)


        out += identity
        out = self.relu(out)

        return out

In [12]:
class AttnBasicBlock(nn.Module):
    
    expansion: int = 1

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, 
                 dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None, attention: bool = True,) -> None:
        super(AttnBasicBlock, self).__init__()
        self.attention = attention

        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:

        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)


        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class BN_layer(nn.Module):
    def __init__(self, block: Type[Union[BasicBlock, Bottleneck]], layers: int, groups: int = 1, width_per_group: int = 64, 
                 norm_layer: Optional[Callable[..., nn.Module]] = None,):
        
        super(BN_layer, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.groups = groups
        self.base_width = width_per_group
        self.inplanes = 256 * block.expansion
        self.dilation = 1
        self.bn_layer = self._make_layer(block, 512, layers, stride=2)

        self.conv1 = conv3x3(64 * block.expansion, 128 * block.expansion, 2)
        self.bn1 = norm_layer(128 * block.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(128 * block.expansion, 256 * block.expansion, 2)
        self.bn2 = norm_layer(256 * block.expansion)
        self.conv3 = conv3x3(128 * block.expansion, 256 * block.expansion, 2)
        self.bn3 = norm_layer(256 * block.expansion)

        self.conv4 = conv1x1(1024 * block.expansion, 512 * block.expansion, 1)
        self.bn4 = norm_layer(512 * block.expansion)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int, stride: int = 1, dilate: bool = False) -> nn.Sequential:
        
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(conv1x1(self.inplanes*3, planes * block.expansion, stride), norm_layer(planes * block.expansion),)

        layers = []
        layers.append(block(self.inplanes*3, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer))
        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        l1 = self.relu(self.bn2(self.conv2(self.relu(self.bn1(self.conv1(x[0]))))))
        l2 = self.relu(self.bn3(self.conv3(x[1])))
        feature = torch.cat([l1,l2,x[2]],dim = 1)
        output = self.bn_layer(feature)

        return output.contiguous()

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [13]:
def resnet18(pretrained: bool = False, progress: bool = True,**kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress, **kwargs), BN_layer(AttnBasicBlock,2,**kwargs)


def resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', BasicBlock, [3, 4, 6, 3], pretrained, progress, **kwargs), BN_layer(AttnBasicBlock,3,**kwargs)


def resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs), BN_layer(AttnBottleneck,3,**kwargs)


def resnet101(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet101', Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs), BN_layer(AttnBasicBlock,3,**kwargs)


def resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet152', Bottleneck, [3, 8, 36, 3], pretrained, progress, **kwargs), BN_layer(AttnBottleneck,3,**kwargs)


def resnext50_32x4d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _resnet('resnext50_32x4d', Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def resnext101_32x8d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _resnet('resnext101_32x8d', Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)


def wide_resnet50_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet50_2', Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs), BN_layer(AttnBottleneck,3,**kwargs)


def wide_resnet101_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _resnet('wide_resnet101_2', Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs), BN_layer(AttnBottleneck,3,**kwargs)



### Decoder Resnet

In [14]:
# decoder
class de_BasicBlock(nn.Module):
    
    expansion: int = 1

    def __init__(self, inplanes: int, planes: int, stride: int = 1, upsample: Optional[nn.Module] = None, groups: int = 1,
                 base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        
        super(de_BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        if stride == 2:
            self.conv1 = deconv2x2(inplanes+2, planes, stride)
            self.cord = CoordConv2d()
            self.cord2 = CoordConv2d()
        else:
            self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.upsample = upsample
        self.stride = stride
        
        
    def forward(self, x: Tensor) -> Tensor:
        
        identity = x
        
        if self.cord is not None:
            out = self.cord.forward(out)
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)

        if self.upsample is not None:
            x = self.cord2.forward(x)
            identity = self.upsample(x)

        out += identity
        out = self.relu(out)

        return out


class de_Bottleneck(nn.Module):

    expansion: int = 4

    def __init__(self, inplanes: int, planes: int, stride: int = 1, upsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
       
        super(de_Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        if stride == 2:
            self.conv2 = deconv2x2(width + 2, width, stride, groups, dilation) # new
            self.cord = CoordConv2d()
            self.cord2 = CoordConv2d()
        else:
            self.conv2 = conv3x3(width, width, stride, groups, dilation)
            self.cord = None
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.upsample = upsample
        self.stride = stride
        
        

    def forward(self, x: Tensor) -> Tensor:

        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        if self.cord is not None:
            out = self.cord.forward(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.upsample is not None:
            x = self.cord2.forward(x)
            identity = self.upsample(x)

        out += identity
        out = self.relu(out)

        return out


class de_ResNet(nn.Module):

    def __init__(self, block: Type[Union[de_BasicBlock, de_Bottleneck]], layers: List[int], num_classes: int = 1000, zero_init_residual: bool = False, groups: int = 1, width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None, norm_layer: Optional[Callable[..., nn.Module]] = None ) -> None:
        
        super(de_ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.inplanes = 512 * block.expansion
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        self.layer1 = self._make_layer(block, 256, layers[0], stride=2)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
 

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, de_Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, de_BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[de_BasicBlock, de_Bottleneck]], planes: int, blocks: int, stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        upsample = None
        previous_dilation = self.dilation
        
        if dilate:
            self.dilation *= stride
            stride = 1
            
        if stride != 1 or self.inplanes != planes * block.expansion: # und here
            upsample = nn.Sequential(deconv2x2(self.inplanes+2, planes * block.expansion, stride),norm_layer(planes * block.expansion),)

        layers = []                       
        layers.append(block(self.inplanes, planes, stride, upsample, self.groups, self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups, base_width=self.base_width, dilation=self.dilation, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:

        # decoer      #change here self. cord()
        feature_a = self.layer1(x)  # 512*8*8->256*16*16
        feature_b = self.layer2(feature_a)  # 256*16*16->128*32*32
        feature_c = self.layer3(feature_b)  # 128*32*32->64*64*64

        return [feature_c, feature_b, feature_a]

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _de_resnet(arch: str, block: Type[Union[de_BasicBlock, de_Bottleneck]], layers: List[int], pretrained: bool, progress: bool, **kwargs: Any) -> ResNet:
    
    
    model = de_ResNet(block, layers, **kwargs)
    
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
        
    return model


In [15]:
def de_resnet18(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _de_resnet('resnet18', de_BasicBlock, [2, 2, 2, 2], pretrained, progress, **kwargs)


def de_resnet34(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _de_resnet('resnet34', de_BasicBlock, [3, 4, 6, 3], pretrained, progress, **kwargs)


def de_resnet50(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _de_resnet('resnet50', de_Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def de_resnet101(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-101 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _de_resnet('resnet101', de_Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)


def de_resnet152(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNet-152 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _de_resnet('resnet152', de_Bottleneck, [3, 8, 36, 3], pretrained, progress, **kwargs)


def de_resnext50_32x4d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-50 32x4d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 4
    return _de_resnet('resnext50_32x4d', de_Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def de_resnext101_32x8d(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""ResNeXt-101 32x8d model from
    `"Aggregated Residual Transformation for Deep Neural Networks" <https://arxiv.org/pdf/1611.05431.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['groups'] = 32
    kwargs['width_per_group'] = 8
    return _de_resnet('resnext101_32x8d', de_Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)


def de_wide_resnet50_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-50-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _de_resnet('wide_resnet50_2', de_Bottleneck, [3, 4, 6, 3], pretrained, progress, **kwargs)


def de_wide_resnet101_2(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> ResNet:
    r"""Wide ResNet-101-2 model from
    `"Wide Residual Networks" <https://arxiv.org/pdf/1605.07146.pdf>`_.
    The model is the same as ResNet except for the bottleneck number of channels
    which is twice larger in every block. The number of channels in outer 1x1
    convolutions is the same, e.g. last block in ResNet-50 has 2048-512-2048
    channels, and in Wide ResNet-50-2 has 2048-1024-2048.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    kwargs['width_per_group'] = 64 * 2
    return _de_resnet('wide_resnet101_2', de_Bottleneck, [3, 4, 23, 3], pretrained, progress, **kwargs)

## Conv

In [16]:
class AddCoords(nn.Module):
    
    def __init__(self, rank, with_r=False, use_cuda=True):
        super(AddCoords, self).__init__()
        self.rank = rank
        self.use_cuda = use_cuda
        

    def forward(self, input_tensor):
        """
        :param input_tensor: shape (N, C_in, H, W)
        :return:
        """
        if self.rank == 2:
            batch_size_shape, channel_in_shape, dim_y, dim_x = input_tensor.shape
            xx_ones = torch.ones([1, 1, 1, dim_x], dtype=torch.int32)
            yy_ones = torch.ones([1, 1, 1, dim_y], dtype=torch.int32)
            

            xx_range = torch.arange(dim_y, dtype=torch.int32)
            yy_range = torch.arange(dim_x, dtype=torch.int32)
            xx_range = xx_range[None, None, :, None]
            yy_range = yy_range[None, None, :, None]

            xx_channel = torch.matmul(xx_range, xx_ones)
            yy_channel = torch.matmul(yy_range, yy_ones)

            # transpose y
            yy_channel = yy_channel.permute(0, 1, 3, 2)

            xx_channel = xx_channel.float() / (dim_y - 1)
            yy_channel = yy_channel.float() / (dim_x - 1)

            xx_channel = xx_channel * 2 - 1
            yy_channel = yy_channel * 2 - 1

            xx_channel = xx_channel.repeat(batch_size_shape, 1, 1, 1)
            yy_channel = yy_channel.repeat(batch_size_shape, 1, 1, 1)

            if torch.cuda.is_available and self.use_cuda:
                input_tensor = input_tensor.cuda()
                xx_channel = xx_channel.cuda()
                yy_channel = yy_channel.cuda()
                out = torch.cat([input_tensor, xx_channel, yy_channel], dim=1)

        else:
            raise NotImplementedError

        return out




class CoordConv2d():
    def __init__(self, with_r=False, use_cuda=True):
        
        super(CoordConv2d, self).__init__()
        self.rank = 2
        self.addcoords = AddCoords(self.rank, with_r, use_cuda=use_cuda)


    def forward(self, input_tensor):
        """
        input_tensor_shape: (N, C_in,H,W)
        output_tensor_shape: N,C_out,H_out,W_out）
        :return: CoordConv2d Result
        """
        out = self.addcoords(input_tensor)

        return out

# Loss

In [17]:
def loss_fucntion(a, b):
    #mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    for item in range(len(a)):
        #print(a[item].shape)
        #print(b[item].shape)
        #loss += 0.1*mse_loss(a[item], b[item])
        loss += torch.mean(1-cos_loss(a[item].view(a[item].shape[0],-1), b[item].view(b[item].shape[0],-1)))
    return loss

def loss_concat(a, b):
    mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    a_map = []
    b_map = []
    size = a[0].shape[-1]
    for item in range(len(a)):
        #loss += mse_loss(a[item], b[item])
        a_map.append(F.interpolate(a[item], size=size, mode='bilinear', align_corners=True))
        b_map.append(F.interpolate(b[item], size=size, mode='bilinear', align_corners=True))
    a_map = torch.cat(a_map,1)
    b_map = torch.cat(b_map,1)
    loss += torch.mean(1-cos_loss(a_map,b_map))
    return loss

# Train

In [18]:
def show_metrics_k_fold(history, metric = "", log = True, modus_min = True):

    print("")
    # inits params
    max_ = 1e6
    num_epochs = len(history["Fold_01"][metric])
    k = len(history)
    
    if num_epochs > 50:
        stepsize = 10
    else:
        stepsize = 5
    range_ =  [max(i-1,0) for i in range(0, num_epochs+1, stepsize)]
    
    # inits arrays
    train_min  = list(np.ones(num_epochs) * max_)
    val_min    = list(np.ones(num_epochs) * max_)
    train_max  = list(np.zeros(num_epochs))
    val_max    = list(np.zeros(num_epochs))
        
    train_mean_array = np.zeros((k, num_epochs))
    val_mean_array  = np.zeros((k, num_epochs))
    
    list_ext_tr = []
    list_ext_val = []
    
    metric_name = get_name(metric)
    
    metric_tr = metric
    metric_val = metric.replace("_tr", "_val")
  
  
    # history[k_fold] -> (list_train_loss, list_val_loss...) 
    
    plt.figure(figsize=(18,6))
                      
        
    # loop over folds
    for j in range(k):
                
        k_fold = "Fold_" + str(("{:02d}".format(j+1)))
        list_tr = history[k_fold][metric_tr]
        list_val = history[k_fold][metric_val]
                
        # find min max and mean for train and val along all folds
        train_min = np.minimum(list_tr, train_min)
        train_max = np.maximum(list_tr, train_max)
                
        val_min   = np.minimum(list_val, val_min)
        val_max   = np.maximum(list_val, val_max)
                
        train_mean_array[j,:]    = list_tr
        val_mean_array[j,:]      = list_val
        
        # for stats: get the mins of all folds 
        if modus_min:
            ind_ext_val = np.argmin(list_val)
        else:
            ind_ext_val = np.argmax(list_val)
                              
        list_ext_tr.append(list_tr[ind_ext_val])
        list_ext_val.append(list_val[ind_ext_val])
                
                
        # plot every metric per fold
        plt.plot(list_tr,   color = 'red',  linestyle = ':') 
        plt.plot(list_val,   color = 'deepskyblue', linestyle = ':')
                
            
    # getting mean value per metric for every epoch
    train_mean = np.mean(train_mean_array, axis = 0)
    val_mean   = np.mean(val_mean_array, axis = 0)
                  
    plt.plot(train_max,  color='crimson', label="train") 
    plt.plot(train_mean, color='darkred', linewidth=5)  
    plt.plot(train_min,  color='crimson')
            
    plt.plot(val_max,   color='dodgerblue', label = "val")
    plt.plot(val_mean,  color='darkblue',  linewidth=5)
    plt.plot(val_min,   color='dodgerblue')
            
    plt.fill_between(range(num_epochs), val_max,  val_min,    alpha=0.3, color='skyblue')
    plt.fill_between(range(num_epochs), train_max, train_min, alpha=0.3, color='pink')
            
    plt.title(f'{metric_name}  vs. epochs', fontsize = 20)
    plt.ylabel(metric_name, fontsize = 17)
    plt.xlabel('epoch', fontsize = 17)
    
    if log:
        plt.yscale('log')
        
    plt.xticks(range(num_epochs), [str(i+1)  if i in range_ else ''  for i in range(num_epochs)]) 
        
    if metric == "loss":
        plt.legend(loc='upper right', frameon = False)
    else:
        plt.legend(loc='upper left', frameon = False)
        
    tr_mean = round(np.mean(list_ext_tr),4)
    tr_min = round(np.min(list_ext_tr), 4)
    tr_max = round(np.max(list_ext_tr),4)
    tr_std = round(np.std(list_ext_tr),4)
    
    val_mean = round(np.mean(list_ext_val),4)
    val_min = round(np.min(list_ext_val), 4)
    val_max = round(np.max(list_ext_val),4)
    val_std = round(np.std(list_ext_val),4)
    
        
    resize = 192
    fontsize = 12
    
    plt.savefig(output_folder + "k_fold_results_" + metric_name  + ".pdf")
    
    if plot_metrics:
        plt.show()
       
    # print additional results for last epoch:
    print("")
    print("Results for best epoch for", metric_name,": ")
    print("Best epoch for  (val):  ", ind_ext_val+1)
    print("Train - Mean: " , tr_mean, pm, tr_std ," - max: ",  tr_max,  " - min: ",  tr_min)
    print("Val  -  Mean:  " , val_mean, pm, val_std,  " - max: ",  val_max,  " - min: ",  val_min)
    print("\n")
    
    return  tr_mean, tr_std, val_mean, val_std, metric_name

In [19]:
def train(_class_, seed):
    
    print("")
    print("class: ", _class_)
    print("")
    
    epochs = 200     #1#200#50 # 20
    learning_rate = 0.005
    batch_size = 16
    image_size = 256
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    data_transforms, gt_transforms, data_transforms_train = get_data_transforms(image_size, image_size)
    
    train_path = '/kaggle/input/mvtec_anomaly_detection/' + _class_ + "/train" 
    print("train path:", train_path)
    
    test_path = '/kaggle/input/mvtec_anomaly_detection/' + _class_
    print("test path:", test_path)
    print("")
        
    train_data = ImageFolder(root=train_path, transform=data_transforms_train)
    test_data = MVTecDataset(root=test_path, transform=data_transforms, gt_transform=gt_transforms, phase="test")
    
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    
   # m = nn.AvgPool2d((2, 2), stride=(2, 2))

    # loading model
    encoder, bn = wide_resnet50_2(pretrained=True)
    encoder = encoder.to(device)
    
    bn = bn.to(device)
    
    encoder.eval()
    
    decoder = de_wide_resnet50_2(pretrained=False)
    decoder = decoder.to(device)

    #optimizer = torch.optim.Adam(list(decoder.parameters())+list(bn.parameters()), lr=learning_rate, betas=(0.5,0.999))
    optimizer = torch.optim.AdamW(list(decoder.parameters())+list(bn.parameters()), lr=learning_rate, betas=(0.5,0.999), weight_decay = 0.01)

    c = 0
    print("len of images in train:", len(glob.glob(train_path + "/good/*.png")))
    print("")
    for dir_ in glob.glob(test_path + "/test/*/", recursive=True):
        print("len of images in test:", dir_,  len(glob.glob(dir_ + "/*.png", recursive=True)))
        c = c + len(glob.glob(dir_ + "/*.png", recursive=True))
    print("ges: ", c)
    print("")
    
    loss_mean_list = []
    loss_mean_list_val = []
    auroc_px_list, auroc_sp_list, aupro_px_list = [],[],[] 
    items_list = []
    
    
    for epoch in range(epochs):
        
        bn.train()
        decoder.train()
        loss_list = []
        
        items_list.append(class_)
        
        
        for i, (img, label) in enumerate(train_dataloader):
            
            #if i == 0:
            #    img_train = img
             #   img_train = m.forward(img_train)
              #  img_train = m.forward(img_train)
               # img_train = m.forward(img_train)
               # img_train = m.forward(img_train)
               # img_train = m.forward(img_train)
               # print(img_train.shape)
            
            #print("img input: ")
            #print(img.shape)
            #plt.imshow(img[0].reshape((img.shape[2],img.shape[3],img.shape[1])))
            #plt.show()
            
            #if i == 0:
            #    img_train = img
            #if img.shape[0] < batch_size:
            #    img_train = img_train[0:img.shape[0]]
            #img = (img +  img_train)/2 
            
            img = img.to(device)
            inputs = encoder(img)
            #print("shape of encoder: ", len(inputs))
            #print(bn(inputs).shape)
            
            outputs = decoder(bn(inputs))#bn(inputs))
            loss = loss_fucntion(inputs, outputs)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
       
        loss_mean_list.append(np.mean(loss_list))
        
        print('epoch {}/{}: loss: {:.4f}'.format(epoch + 1, epochs, np.mean(loss_list)))
        
        if (epoch + 1) % 1 == 0:
            
            auroc_px, auroc_sp, aupro_px, loss_val = evaluation(encoder, bn, decoder, test_dataloader, device, plot = False)
            print('Pixel Auroc: {:.4f}, Sample Auroc: {:.4f}, Pixel Aupro: {:.4}, Loss (Good): {:.4}'.format(auroc_px, auroc_sp, aupro_px, loss_val))
            print("")
            
            auroc_px_list.append(auroc_px)
            auroc_sp_list.append(auroc_sp)
            aupro_px_list.append(aupro_px)
            loss_mean_list_val.append(loss_val)
            
    
    log = False
    linewidth = 2
    
    f, ax = plt.subplots(1,1, figsize=(10,4))
    ax2 = ax.twinx()
    ax.plot(np.arange(len(loss_mean_list)), loss_mean_list, label = "train loss", color = "red",  linewidth = linewidth)
    ax.plot(np.arange(len(loss_mean_list_val)), loss_mean_list_val, label = "val loss", color = "black",  linewidth = linewidth)
    
    ax2.plot(np.arange(len(auroc_px_list)), auroc_px_list, label = "validation auroc px", color = "blue",  linewidth = linewidth)
    ax2.plot(np.arange(len(auroc_sp_list)), auroc_sp_list, label = "validation auroc sp", color = "purple",  linewidth = linewidth)
    ax2.plot(np.arange(len(aupro_px_list)), aupro_px_list, label = "validation aupro px", color = "cyan",  linewidth = linewidth)
    

    #ax.legend(frameon = False)
    ax2.legend(frameon = False)
    if log:
        ax.set_yscale('log')
        ax2.set_yscale('log')
    #ax.set_xticks(np.arange(1, len(loss_mean_list)+1))
    #ax2.set_xticks(np.arange(1, len(loss_mean_list)+1))
    plt.show()
    plt.close()
    
    
    sum_ = np.array(auroc_px_list) + np.array(aupro_px_list)
    
    
    d_class = {'items': items_list, 'loss': loss_mean_list,  'auroc_px': auroc_px_list,'auroc_sp': auroc_sp_list,'aupro_px': aupro_px_list, 'loss_val': loss_mean_list_val}

    df = pd.DataFrame(data=d_class)

    df.to_csv('df_' + class_ + "_" + str(seed) + "_.csv", index=False) 
    
    
    print("LOSS: ", np.round(np.min(loss_mean_list),4), " at epoch: ", np.argmin(loss_mean_list)+1)
    print("AUROC PX: ", np.max(auroc_px_list), " at epoch: ", np.argmax(auroc_px_list)+1)
    print("AUROC SP: ", np.max(auroc_sp_list), " at epoch: ", np.argmax(auroc_sp_list)+1)
    print("AUPRO PX: ", np.max(aupro_px_list), " at epoch: ", np.argmax(aupro_px_list)+1)
    
    print("LOSS VAL: ", np.round(np.min(loss_mean_list_val),4), " at epoch: ", np.argmin(loss_mean_list_val)+1)
    print("AUROC PX: ", auroc_px_list[np.argmin(loss_mean_list_val)], " at epoch: ",np.argmin(loss_mean_list_val)+1)
    print("AUROC SP: ", auroc_sp_list[np.argmin(loss_mean_list_val)], " at epoch: ", np.argmin(loss_mean_list_val)+1)
    print("AUPRO PX: ", aupro_px_list[np.argmin(loss_mean_list_val)], " at epoch: ", np.argmin(loss_mean_list_val)+1)
    
    print("")
    print("sum:")
    print("AUROC PX: ", auroc_px_list[np.argmax(sum_)], " at epoch: ", np.argmax(sum_)+1)
    print("AUPRO PX: ", aupro_px_list[np.argmax(sum_)], " at epoch: ", np.argmax(sum_)+1)

            
    return np.max(auroc_px_list), np.max(auroc_sp_list), np.max(aupro_px_list)

### Test - Evaluation

In [20]:
def cal_anomaly_map(fs_list, ft_list, out_size=224, amap_mode='mul'):
    
    if amap_mode == 'mul':
        anomaly_map = np.ones([out_size, out_size])
    else:
        anomaly_map = np.zeros([out_size, out_size])
        
    a_map_list = []
    
    for i in range(len(ft_list)): # list mit den 3 maps
        fs = fs_list[i]
        ft = ft_list[i]

        a_map = 1 - F.cosine_similarity(fs, ft) # torch.Size([1, 64, 64]) , torch.Size([1, 32, 32]),  torch.Size([1, 16, 16])
        a_map = torch.unsqueeze(a_map, dim=1) # eins davor in shape
        a_map = F.interpolate(a_map, size=out_size, mode='bilinear', align_corners=True) # -> alle auf torch.Size([1, 1, 256, 256])
        a_map = a_map[0, 0, :, :].to('cpu').detach().numpy()
        a_map_list.append(a_map)
        
        if amap_mode == 'mul':
            anomaly_map *= a_map
        else:
            anomaly_map += a_map
            
    return anomaly_map, a_map_list


def show_cam_on_image(img, anomaly_map):
    cam = np.float32(anomaly_map)/255 + np.float32(img)/255
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

def min_max_norm(image):
    a_min, a_max = image.min(), image.max()
    return (image-a_min)/(a_max - a_min)

def cvt2heatmap(gray):
    heatmap = cv2.applyColorMap(np.uint8(gray), cv2.COLORMAP_JET)
    return heatmap



def evaluation(encoder, bn, decoder, dataloader, device, _class_=None, plot = False):

    bn.eval()
    decoder.eval()

    gt_list_px = []
    pr_list_px = []
    gt_list_sp = []
    pr_list_sp = []
    aupro_list = []
    loss_list_val = []
    
    with torch.no_grad():
        for img, gt, label, typ in dataloader:

            
            img = img.to(device)
            inputs = encoder(img)
            outputs = decoder(bn(inputs))
            
            loss_val = loss_fucntion(inputs, outputs)
            
            if typ[0] == "good": 
                loss_list_val.append(loss_val.item())
            
            #print("inputs len ", len(inputs), "shape: ", inputs[0].shape,  inputs[1].shape,  inputs[2].shape)
            #print("outputs len ", len(outputs), "shape: ", outputs[0].shape,  outputs[1].shape,  outputs[2].shape)
            
            #inputs len  3 shape:  torch.Size([1, 64, 64, 64]) torch.Size([1, 128, 32, 32]) torch.Size([1, 256, 16, 16])
            #outputs len  3 shape:  torch.Size([1, 64, 64, 64]) torch.Size([1, 128, 32, 32]) torch.Size([1, 256, 16, 16])
            
            anomaly_map, _ = cal_anomaly_map(inputs, outputs, img.shape[-1], amap_mode='a') # -> snomaly map (256, 256)
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            
            
            gt[gt > 0.5] = 1
            gt[gt <= 0.5] = 0
            
            if label.item()!=0:
                aupro_list.append(compute_pro(gt.squeeze(0).cpu().numpy().astype(int), anomaly_map[np.newaxis,:,:]))
                
            gt_list_px.extend(gt.cpu().numpy().astype(int).ravel())
            pr_list_px.extend(anomaly_map.ravel())
            gt_list_sp.append(np.max(gt.cpu().numpy().astype(int)))
            pr_list_sp.append(np.max(anomaly_map))
            
            #print("anomaly map" , anomaly_map.shape)
            #print(gt.shape)
            
            if plot:
                f, ax = plt.subplots(1,5, figsize=(10,4))

                sas = img.detach().cpu().numpy()
                ax[2].imshow(anomaly_map)
                ax[0].imshow(sas[0].transpose(1, 2, 0))
                ax[1].imshow(gt[0,0,:,:])

                ano_map = min_max_norm(anomaly_map)
                ano_map = cvt2heatmap(ano_map * 255)

                ax[3].imshow(ano_map)

                ano_map[ano_map > 0.5] = 1
                ano_map[ano_map <= 0.5] = 0

                ax[4].imshow(ano_map)
                plt.axis('off')
                plt.show()

                try:
                    #print(round(roc_auc_score(gt.cpu().numpy().astype(int).ravel(), anomaly_map.ravel()), 4))
                    print(compute_pro(gt.squeeze(0).cpu().numpy().astype(int), anomaly_map[np.newaxis,:,:]))
                except:
                    print("")
            
            #print(len(gt_list_px))
            #print(len(pr_list_px))
            #print(len(gt_list_sp))
            #print(len(pr_list_sp))
            #print(gt_list_sp)
            #print(pr_list_sp)
            

        #ano_score = (pr_list_sp - np.min(pr_list_sp)) / (np.max(pr_list_sp) - np.min(pr_list_sp))
        #vis_data = {}
        #vis_data['Anomaly Score'] = ano_score
        #vis_data['Ground Truth'] = np.array(gt_list_sp)
        # print(type(vis_data))
        # np.save('vis.npy',vis_data)
        #with open('{}_vis.pkl'.format(_class_), 'wb') as f:
        #    pickle.dump(vis_data, f, pickle.HIGHEST_PROTOCOL)

        auroc_px = round(roc_auc_score(gt_list_px, pr_list_px), 4)
        auroc_sp = round(roc_auc_score(gt_list_sp, pr_list_sp), 4)
        
        loss_val = np.mean(loss_list_val)
        
        #print(loss_list_val)
        
    return auroc_px, auroc_sp, round(np.mean(aupro_list),4), loss_val




def compute_pro(masks: ndarray, amaps: ndarray, num_th: int = 200) -> None:

    """Compute the area under the curve of per-region overlaping (PRO) and 0 to 0.3 FPR
    Args:
        category (str): Category of product
        masks (ndarray): All binary masks in test. masks.shape -> (num_test_data, h, w)
        amaps (ndarray): All anomaly maps in test. amaps.shape -> (num_test_data, h, w)
        num_th (int, optional): Number of thresholds
    """
    assert isinstance(amaps, ndarray), "type(amaps) must be ndarray"
    assert isinstance(masks, ndarray), "type(masks) must be ndarray"
    assert amaps.ndim == 3, "amaps.ndim must be 3 (num_test_data, h, w)"
    assert masks.ndim == 3, "masks.ndim must be 3 (num_test_data, h, w)"
    assert amaps.shape == masks.shape, "amaps.shape and masks.shape must be same"
    assert set(masks.flatten()) == {0, 1}, "set(masks.flatten()) must be {0, 1}"
    assert isinstance(num_th, int), "type(num_th) must be int"

    df = pd.DataFrame([], columns=["pro", "fpr", "threshold"])
    binary_amaps = np.zeros_like(amaps, dtype = bool)

    min_th = amaps.min()
    max_th = amaps.max()
    delta = (max_th - min_th) / num_th
    

    for th in np.arange(min_th, max_th, delta):
        
        binary_amaps[amaps <= th] = 0
        binary_amaps[amaps > th] = 1

        pros = []
        
        for binary_amap, mask in zip(binary_amaps, masks):
            for region in measure.regionprops(measure.label(mask)):
                       
                
                axes0_ids = region.coords[:, 0]
                axes1_ids = region.coords[:, 1]
                tp_pixels = binary_amap[axes0_ids, axes1_ids].sum()
                
                #print(tp_pixels, region.area)
                #print((mask==1).sum())
                #print(binary_amap[mask==1].sum())
                
                pros.append(tp_pixels / region.area) # tpr wird hier als overlap definiert, sind mehrere regionen wird gemittelt deswegen unten mean(()

        inverse_masks = 1 - masks
        fp_pixels = np.logical_and(inverse_masks, binary_amaps).sum()
        fpr = fp_pixels / inverse_masks.sum()

        #print(fpr, mean(pros), th)
        df_new = pd.DataFrame({"pro": mean(pros), "fpr": fpr, "threshold": th}, index=[0])
        df = pd.concat([df if not df.empty else None, df_new], ignore_index=True)
        #print(df)
        #print("---")

    # Normalize FPR from 0 ~ 1 to 0 ~ 0.3
    #print(len(df))
    df = df[df["fpr"] < 0.3] # aktives filtern und dann back scalen to 0-1
    #print(len(df))
    df["fpr"] = df["fpr"] / df["fpr"].max()

    pro_auc = auc(df["fpr"], df["pro"])    

    return pro_auc

def detection(encoder, bn, decoder, dataloader,device,_class_):
    #_, t_bn = resnet50(pretrained=True)
    bn.load_state_dict(bn.state_dict())
    bn.eval()
    #t_bn.to(device)
    #t_bn.load_state_dict(bn.state_dict())
    decoder.eval()
    gt_list_sp = []
    prmax_list_sp = []
    prmean_list_sp = []
    with torch.no_grad():
        for img, label in dataloader:

            img = img.to(device)
            if img.shape[1] == 1:
                img = img.repeat(1, 3, 1, 1)
            label = label.to(device)
            inputs = encoder(img)
            outputs = decoder(bn(inputs))
            anomaly_map, _ = cal_anomaly_map(inputs, outputs, img.shape[-1], 'acc')
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)


            gt_list_sp.extend(label.cpu().data.numpy())
            prmax_list_sp.append(np.max(anomaly_map))
            prmean_list_sp.append(np.sum(anomaly_map))#np.sum(anomaly_map.ravel().argsort()[-1:][::-1]))

        gt_list_sp = np.array(gt_list_sp)
        indx1 = gt_list_sp == _class_
        indx2 = gt_list_sp != _class_
        gt_list_sp[indx1] = 0
        gt_list_sp[indx2] = 1


        auroc_sp_max = round(roc_auc_score(gt_list_sp, prmax_list_sp), 4)
        auroc_sp_mean = round(roc_auc_score(gt_list_sp, prmean_list_sp), 4)
        
    return auroc_sp_max, auroc_sp_mean

In [21]:
def test(_class_):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)
    print(_class_)

    data_transform, gt_transform = get_data_transforms(256, 256)
    test_path = '../mvtec/' + _class_
    ckp_path = './checkpoints/' + 'rm_1105_wres50_ff_mm_' + _class_ + '.pth'
    test_data = MVTecDataset(root=test_path, transform=data_transform, gt_transform=gt_transform, phase="test")
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    encoder, bn = wide_resnet50_2(pretrained=True)
    encoder = encoder.to(device)
    bn = bn.to(device)
    encoder.eval()
    decoder = de_wide_resnet50_2(pretrained=False)
    decoder = decoder.to(device)
    ckp = torch.load(ckp_path)
    for k, v in list(ckp['bn'].items()):
        if 'memory' in k:
            ckp['bn'].pop(k)
    decoder.load_state_dict(ckp['decoder'])
    bn.load_state_dict(ckp['bn'])
    auroc_px, auroc_sp, aupro_px = evaluation(encoder, bn, decoder, test_dataloader, device,_class_)
    print(_class_,':',auroc_px,',',auroc_sp,',',aupro_px)
    return auroc_px

In [22]:
def visualization(_class_):

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    data_transform, gt_transform = get_data_transforms(256, 256)
    test_path = '../mvtec/' + _class_
    ckp_path = './checkpoints/' + 'rm_1105_wres50_ff_mm_'+_class_+'.pth'
    test_data = MVTecDataset(root=test_path, transform=data_transform, gt_transform=gt_transform, phase="test")
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

    encoder, bn = wide_resnet50_2(pretrained=True)
    encoder = encoder.to(device)
    bn = bn.to(device)

    encoder.eval()
    decoder = de_wide_resnet50_2(pretrained=False)
    decoder = decoder.to(device)
    ckp = torch.load(ckp_path)
    for k, v in list(ckp['bn'].items()):
        if 'memory' in k:
            ckp['bn'].pop(k)
    decoder.load_state_dict(ckp['decoder'])
    bn.load_state_dict(ckp['bn'])

    count = 0
    with torch.no_grad():
        for img, gt, label, _ in test_dataloader:
            if (label.item() == 0):
                continue
            #if count <= 10:
            #    count += 1
            #    continue

            decoder.eval()
            bn.eval()

            img = img.to(device)
            inputs = encoder(img)
            outputs = decoder(bn(inputs))

            #inputs.append(feature)
            #inputs.append(outputs)
            #t_sne(inputs)


            anomaly_map, amap_list = cal_anomaly_map([inputs[-1]], [outputs[-1]], img.shape[-1], amap_mode='a')
            anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            ano_map = min_max_norm(anomaly_map)
            ano_map = cvt2heatmap(ano_map*255)
            img = cv2.cvtColor(img.permute(0, 2, 3, 1).cpu().numpy()[0] * 255, cv2.COLOR_BGR2RGB)
            img = np.uint8(min_max_norm(img)*255)
            #if not os.path.exists('./results_all/'+_class_):
            #    os.makedirs('./results_all/'+_class_)
            #cv2.imwrite('./results_all/'+_class_+'/'+str(count)+'_'+'org.png',img)
            #plt.imshow(img)
            #plt.axis('off')
            #plt.savefig('org.png')
            #plt.show()
            ano_map = show_cam_on_image(img, ano_map)
            #cv2.imwrite('./results_all/'+_class_+'/'+str(count)+'_'+'ad.png', ano_map)
            plt.imshow(ano_map)
            plt.axis('off')
            #plt.savefig('ad.png')
            plt.show()

            gt = gt.cpu().numpy().astype(int)[0][0]*255
            #cv2.imwrite('./results/'+_class_+'_'+str(count)+'_'+'gt.png', gt)

            #b, c, h, w = inputs[2].shape
            #t_feat = F.normalize(inputs[2], p=2).view(c, -1).permute(1, 0).cpu().numpy()
            #s_feat = F.normalize(outputs[2], p=2).view(c, -1).permute(1, 0).cpu().numpy()
            #c = 1-min_max_norm(cv2.resize(anomaly_map,(h,w))).flatten()
            #print(c.shape)
            #t_sne([t_feat, s_feat], c)
            #assert 1 == 2

            #name = 0
            #for anomaly_map in amap_list:
            #    anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            #    ano_map = min_max_norm(anomaly_map)
            #    ano_map = cvt2heatmap(ano_map * 255)
                #ano_map = show_cam_on_image(img, ano_map)
                #cv2.imwrite(str(name) + '.png', ano_map)
                #plt.imshow(ano_map)
                #plt.axis('off')
                #plt.savefig(str(name) + '.png')
                #plt.show()
            #    name+=1
            count += 1
            #if count>20:
            #    return 0
                #assert 1==2


def vis_nd(name, _class_):
    print(name,':',_class_)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    ckp_path = './checkpoints/' + name + '_' + str(_class_) + '.pth'
    train_dataloader, test_dataloader = load_data(name, _class_, batch_size=16)

    encoder, bn = resnet18(pretrained=True)
    encoder = encoder.to(device)
    bn = bn.to(device)
    encoder.eval()
    decoder = de_resnet18(pretrained=False)
    decoder = decoder.to(device)

    ckp = torch.load(ckp_path)

    decoder.load_state_dict(ckp['decoder'])
    bn.load_state_dict(ckp['bn'])
    decoder.eval()
    bn.eval()

    gt_list_sp = []
    prmax_list_sp = []
    prmean_list_sp = []

    count = 0
    with torch.no_grad():
        for img, label in test_dataloader:
            if img.shape[1] == 1:
                img = img.repeat(1, 3, 1, 1)
            #if count <= 10:
            #    count += 1
            #    continue
            img = img.to(device)
            inputs = encoder(img)
            #print(inputs[-1].shape)
            outputs = decoder(bn(inputs))


            anomaly_map, amap_list = cal_anomaly_map(inputs, outputs, img.shape[-1], amap_mode='a')
            #anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            ano_map = min_max_norm(anomaly_map)
            ano_map = cvt2heatmap(ano_map*255)
            img = cv2.cvtColor(img.permute(0, 2, 3, 1).cpu().numpy()[0] * 255, cv2.COLOR_BGR2RGB)
            img = np.uint8(min_max_norm(img)*255)
            cv2.imwrite('./nd_results/'+name+'_'+str(_class_)+'_'+str(count)+'_'+'org.png',img)
            #plt.imshow(img)
            #plt.axis('off')
            #plt.savefig('org.png')
            #plt.show()
            ano_map = show_cam_on_image(img, ano_map)
            cv2.imwrite('./nd_results/'+name+'_'+str(_class_)+'_'+str(count)+'_'+'ad.png', ano_map)
            #plt.imshow(ano_map)
            #plt.axis('off')
            #plt.savefig('ad.png')
            #plt.show()

            #gt = gt.cpu().numpy().astype(int)[0][0]*255
            #cv2.imwrite('./results/'+_class_+'_'+str(count)+'_'+'gt.png', gt)

            #b, c, h, w = inputs[2].shape
            #t_feat = F.normalize(inputs[2], p=2).view(c, -1).permute(1, 0).cpu().numpy()
            #s_feat = F.normalize(outputs[2], p=2).view(c, -1).permute(1, 0).cpu().numpy()
            #c = 1-min_max_norm(cv2.resize(anomaly_map,(h,w))).flatten()
            #print(c.shape)
            #t_sne([t_feat, s_feat], c)
            #assert 1 == 2

            #name = 0
            #for anomaly_map in amap_list:
            #    anomaly_map = gaussian_filter(anomaly_map, sigma=4)
            #    ano_map = min_max_norm(anomaly_map)
            #    ano_map = cvt2heatmap(ano_map * 255)
                #ano_map = show_cam_on_image(img, ano_map)
                #cv2.imwrite(str(name) + '.png', ano_map)
                #plt.imshow(ano_map)
                #plt.axis('off')
                #plt.savefig(str(name) + '.png')
                #plt.show()
            #    name+=1
            #count += 1
            #if count>40:
            #    return 0
                #assert 1==2
            gt_list_sp.extend(label.cpu().data.numpy())
            prmax_list_sp.append(np.max(anomaly_map))
            prmean_list_sp.append(np.sum(anomaly_map))  # np.sum(anomaly_map.ravel().argsort()[-1:][::-1]))

        gt_list_sp = np.array(gt_list_sp)
        indx1 = gt_list_sp == _class_
        indx2 = gt_list_sp != _class_
        gt_list_sp[indx1] = 0
        gt_list_sp[indx2] = 1

        ano_score = (prmean_list_sp-np.min(prmean_list_sp))/(np.max(prmean_list_sp)-np.min(prmean_list_sp))
        vis_data = {}
        vis_data['Anomaly Score'] = ano_score
        vis_data['Ground Truth'] = np.array(gt_list_sp)
        #print(type(vis_data))
        #np.save('vis.npy',vis_data)
        with open('vis.pkl','wb') as f:
            pickle.dump(vis_data,f,pickle.HIGHEST_PROTOCOL)



# Main

In [23]:
seed = 111

setup_seed(seed)

#item_list = ['carpet', 'bottle', 'hazelnut', 'leather', 'cable', 'capsule', 'grid', 'pill','transistor', 'metal_nut', 'screw','toothbrush', 'zipper', 'tile', 'wood']

item_list = ['carpet']

list_auroc_px, list_auroc_sp, list_aupro_px = [],[],[]

for class_ in item_list: 
    
    auroc_px, auroc_sp, aupro_px = train(class_, seed)
    
    list_auroc_px.append(auroc_px)
    list_auroc_sp.append(auroc_sp)
    list_aupro_px.append(aupro_px)
    
    print("")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("")
    print("")
    
    
    
d = {'items': item_list, 'auroc_px': list_auroc_px,'auroc_sp': list_auroc_sp,'aupro_px': list_aupro_px}

df = pd.DataFrame(data=d)

df.to_csv('df.csv', index=False) 
    
print("")
print("")
print("FINAL RESULTS:")

print("list_auroc_px: ",np.mean(list_auroc_px))
print("list_auroc_sp: ",np.mean(list_auroc_sp))
print("list_aupro_px: ",np.mean(list_aupro_px))



class:  bottle

train path: /kaggle/input/mvtec_anomaly_detection/bottle/train
test path: /kaggle/input/mvtec_anomaly_detection/bottle



Downloading: "https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth
100%|██████████| 132M/132M [00:00<00:00, 244MB/s] 


len of images in train: 209

len of images in test: /kaggle/input/mvtec_anomaly_detection/bottle/test/good/ 20
len of images in test: /kaggle/input/mvtec_anomaly_detection/bottle/test/contamination/ 21
len of images in test: /kaggle/input/mvtec_anomaly_detection/bottle/test/broken_large/ 20
len of images in test: /kaggle/input/mvtec_anomaly_detection/bottle/test/broken_small/ 22
ges:  83

epoch 1/200: loss: 1.1628


KeyboardInterrupt: 

In [ ]:
seed = 253

setup_seed(seed)

#item_list = ['carpet', 'bottle', 'hazelnut', 'leather', 'cable', 'capsule', 'grid', 'pill','transistor', 'metal_nut', 'screw','toothbrush', 'zipper', 'tile', 'wood']

item_list = ['carpet']

list_auroc_px, list_auroc_sp, list_aupro_px = [],[],[]

for class_ in item_list: 
    
    auroc_px, auroc_sp, aupro_px = train(class_, seed)
    
    list_auroc_px.append(auroc_px)
    list_auroc_sp.append(auroc_sp)
    list_aupro_px.append(aupro_px)
    
    print("")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("")
    print("")
    
    
    
d = {'items': item_list, 'auroc_px': list_auroc_px,'auroc_sp': list_auroc_sp,'aupro_px': list_aupro_px}

df = pd.DataFrame(data=d)

df.to_csv('df.csv', index=False) 
    
print("")
print("")
print("FINAL RESULTS:")

print("list_auroc_px: ",np.mean(list_auroc_px))
print("list_auroc_sp: ",np.mean(list_auroc_sp))
print("list_aupro_px: ",np.mean(list_aupro_px))


In [ ]:
seed = 42

setup_seed(seed)

#item_list = ['carpet', 'bottle', 'hazelnut', 'leather', 'cable', 'capsule', 'grid', 'pill','transistor', 'metal_nut', 'screw','toothbrush', 'zipper', 'tile', 'wood']

item_list = ['carpet']

list_auroc_px, list_auroc_sp, list_aupro_px = [],[],[]

for class_ in item_list: 
    
    auroc_px, auroc_sp, aupro_px = train(class_, seed)
    
    list_auroc_px.append(auroc_px)
    list_auroc_sp.append(auroc_sp)
    list_aupro_px.append(aupro_px)
    
    print("")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("")
    print("")
    
    
    
d = {'items': item_list, 'auroc_px': list_auroc_px,'auroc_sp': list_auroc_sp,'aupro_px': list_aupro_px}

df = pd.DataFrame(data=d)

df.to_csv('df.csv', index=False) 
    
print("")
print("")
print("FINAL RESULTS:")

print("list_auroc_px: ",np.mean(list_auroc_px))
print("list_auroc_sp: ",np.mean(list_auroc_sp))
print("list_aupro_px: ",np.mean(list_aupro_px))
